# Shadow Device Comparison Analysis

This notebook demonstrates the workflow for analyzing shadow device logs and comparing them to production device outputs.

## Setup

First, let's import the necessary modules and check for dependencies.

In [ ]:
# Standard imports
from pathlib import Path

# Shadow device tools
from openpilot.tools.shadow.comparison_logger import ComparisonLogger, FrameData
from openpilot.tools.shadow.align import LogAligner, validate_alignment
from openpilot.tools.shadow.metrics import compute_all_metrics, format_report_markdown

# Visualization
try:
    from openpilot.tools.shadow.visualize import (
        plot_time_series,
        plot_error_histogram,
        plot_control_heatmap,
        plot_correlation_scatter,
        plot_event_timeline,
        plot_summary_dashboard,
        MATPLOTLIB_AVAILABLE
    )
    import matplotlib.pyplot as plt
    print("Visualization libraries available")
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    print("Visualization not available - install matplotlib: pip install matplotlib")

## 1. Generate Sample Data

For this example, we'll generate synthetic shadow and production logs.
In real usage, you would load logs from `/data/shadow_logs/` on your devices.

In [ ]:
import time
import random
import math

random.seed(42)

def generate_sample_frames(device_id: str, n_frames: int = 1000, noise_level: float = 0.05) -> list[FrameData]:
    """Generate sample frames simulating a drive with steering and acceleration."""
    frames = []
    base_time = time.time()
    
    for i in range(n_frames):
        # Simulate a winding road with varying speed
        t = i / 100.0  # 100 Hz
        
        # Base steering: sinusoidal curve
        base_steer = 0.3 * math.sin(t * 0.5) + 0.1 * math.sin(t * 1.5)
        # Base acceleration: varies with road curvature
        base_accel = 0.5 - 0.3 * abs(base_steer)
        
        # Add device-specific noise
        steer = base_steer + random.gauss(0, noise_level)
        accel = base_accel + random.gauss(0, noise_level * 0.5)
        
        # Generate events occasionally
        events = []
        if abs(steer) > 0.25 and random.random() < 0.1:
            events.append("sharpCurve")
        if accel < 0 and random.random() < 0.05:
            events.append("braking")
        
        frame = FrameData(
            frame_id=i,
            timestamp_mono=base_time + t,
            timestamp_gps=base_time + t + random.gauss(0, 0.001),  # Small GPS jitter
            controls={
                "steer_torque": steer,
                "accel": accel,
                "steering_angle_deg": steer * 45,
            },
            model_outputs={
                "desired_curvature": base_steer * 0.01,
            },
            state={
                "v_ego": 25.0 + accel * 5,
                "a_ego": accel,
                "lat_active": True,
                "long_active": True,
            },
            events=events,
        )
        frames.append(frame)
    
    return frames

# Generate sample data
print("Generating sample shadow device frames...")
shadow_frames = generate_sample_frames("shadow", n_frames=1000, noise_level=0.05)

print("Generating sample production device frames...")
production_frames = generate_sample_frames("production", n_frames=1000, noise_level=0.03)

print(f"Shadow frames: {len(shadow_frames)}")
print(f"Production frames: {len(production_frames)}")

## 2. Align Logs

The `LogAligner` synchronizes frames from both devices using GPS timestamps.

In [ ]:
aligner = LogAligner()

# auto_align tries GPS first, then frame ID, then timestamp
result = aligner.auto_align(shadow_frames, production_frames)

print(f"Alignment method: {result.method}")
print(f"Aligned pairs: {len(result.pairs)}")
print(f"Shadow-only frames: {len(result.shadow_only)}")
print(f"Production-only frames: {len(result.production_only)}")
print(f"Mean time offset: {result.mean_time_offset_ms:.2f} ms")
print(f"Alignment quality: {result.alignment_quality:.1%}")

### Validate Alignment Quality

In [ ]:
validation = validate_alignment(result)

print("Alignment Validation")
print("-" * 40)
for key, value in validation.items():
    print(f"{key}: {value}")

## 3. Compute Comparison Metrics

Now we compute detailed metrics comparing shadow and production outputs.

In [ ]:
report = compute_all_metrics(result)

# Display summary
print("Control Metrics")
print("-" * 40)
print(f"Steer RMSE: {report.control_metrics.steer_rmse:.4f}")
print(f"Steer MAE: {report.control_metrics.steer_mae:.4f}")
print(f"Steer Max Error: {report.control_metrics.steer_max_error:.4f}")
print(f"Accel RMSE: {report.control_metrics.accel_rmse:.4f}")
print(f"Accel MAE: {report.control_metrics.accel_mae:.4f}")
print(f"Accel Max Error: {report.control_metrics.accel_max_error:.4f}")

### Generate Markdown Report

In [ ]:
markdown_report = format_report_markdown(report)
print(markdown_report)

## 4. Visualizations

Let's create visualizations to understand the differences between devices.

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Enable inline plotting
    %matplotlib inline
    
    # Time series comparison
    fig = plot_time_series(result.pairs, "steer", title="Steering Comparison")
    plt.show()
else:
    print("Skipping visualizations - matplotlib not available")

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Error distribution
    fig = plot_error_histogram(result.pairs, "steer")
    plt.show()

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Correlation scatter
    fig = plot_correlation_scatter(result.pairs, "steer")
    plt.show()

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Control heatmap
    fig = plot_control_heatmap(result.pairs)
    plt.show()

In [ ]:
if MATPLOTLIB_AVAILABLE:
    # Event timeline
    fig = plot_event_timeline(result.pairs)
    plt.show()

### Summary Dashboard

In [ ]:
if MATPLOTLIB_AVAILABLE:
    fig = plot_summary_dashboard(result, report)
    plt.show()

## 5. Algorithm Harness Integration

Shadow logs can be imported into the algorithm test harness for replay testing.

In [ ]:
from openpilot.selfdrive.controls.lib.tests.algorithm_harness.shadow_import import (
    import_shadow_log,
    compare_shadow_to_harness,
    format_shadow_comparison_report,
)

# Convert shadow frames to harness scenario
scenario = import_shadow_log(
    shadow_frames,
    name="sample_drive",
    mode="lateral",
)

print(f"Scenario: {scenario.name}")
print(f"States: {len(scenario.states)}")
print(f"Description: {scenario.description}")
print(f"Metadata: {scenario.metadata}")

### Simulate Harness Outputs

In practice, you would run an algorithm through the harness.
Here we simulate outputs for demonstration.

In [ ]:
# Simulate harness outputs (in practice, these come from running an algorithm)
harness_outputs = [
    f.controls.get("steer_torque", 0.0) * 0.98 + random.gauss(0, 0.02)  # Simulated algorithm output
    for f in shadow_frames
]

# Compare
metrics = compare_shadow_to_harness(shadow_frames, harness_outputs, mode="lateral")

print("Shadow vs Harness Metrics")
print("-" * 40)
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Generate formatted report
harness_report = format_shadow_comparison_report(metrics, "SimulatedAlgorithm")
print(harness_report)

## 6. Loading Real Logs

When working with real data, use the `ComparisonLogger` to load segments.

In [ ]:
# Example: Load logs from device
# shadow_frames = ComparisonLogger.load_segment("/data/shadow_logs/segment_001")
# production_frames = ComparisonLogger.load_segment("/data/production_logs/segment_001")

print("To load real logs, use:")
print("  shadow_frames = ComparisonLogger.load_segment('/data/shadow_logs/segment_001')")
print("  production_frames = ComparisonLogger.load_segment('/data/production_logs/segment_001')")

## Summary

This notebook demonstrated:

1. **Log Alignment** - Synchronizing frames from shadow and production devices using GPS timestamps
2. **Metric Computation** - Calculating RMSE, MAE, and other comparison metrics
3. **Visualization** - Creating time series, histograms, scatter plots, and dashboards
4. **Algorithm Harness** - Importing shadow logs for replay testing

### Next Steps

- Capture real logs on your shadow device
- Compare different algorithm versions
- Identify scenarios where devices diverge
- Use the harness to test algorithm changes on real data